In [2]:
import pandas as pd
import os
import numpy as np
from bokeh.layouts import gridplot

url_2 = "https://raw.githubusercontent.com/lukes/ISO-3166-Countries-with-Regional-Codes/master/all/all.csv"
regional_file_name = "regions.csv"

file_name = 'WFPVAM_FoodPrices_version5_Retail.csv'

# Add region and datetime cols.
df = pd.read_csv(file_name)
df['date'] = pd.to_datetime(df.date, format='%Y-%m')

# Each product in a graph for each country.

region_df = pd.read_csv(regional_file_name)
region_df.rename(columns={'name': 'adm0_name'}, inplace=True)
new_regions = region_df.loc[:, ['adm0_name', 'sub-region']]

df_regions = pd.merge(df, new_regions, on='adm0_name', how='left')
df = df_regions.copy()

ImportError: cannot import name 'multiarray'

In [49]:
# only_bread.plot(x='date', y='Diff')

def datetime(x):
    return np.array(x, dtype=np.datetime64)


from bokeh.layouts import gridplot
from bokeh.plotting import figure, show, output_file
from bokeh.models import HoverTool


# COLORS
from bokeh.palettes import Dark2_5 as palette
import itertools


colors = itertools.cycle(palette)

# Question 1

Are there any food prices that are show negative/positive correlation, and is this correlation present throughout the years, or perhaps only in certain period? Can you perhaps detect possible ingredients of a certain other food product?



In [50]:
# Each product in a graph for each country.

country = 'Uganda'
y_axis = 'Gradient'

df_country = df.loc[df.adm0_name == country]
plots = []
country_groups = df_country.groupby(['mkt_name'])

# allow for interactive hover tool
# Import the ColumnDataSource
from bokeh.models import ColumnDataSource

for country_group, country_row in country_groups:
    plot = figure(x_axis_type="datetime", title="Products in {}".format(country_group), tools='hover,pan,wheel_zoom,box_zoom,reset')
    product_groups = country_row.groupby(['cm_name'])
    for (group, row), color in zip(product_groups, colors):
        # create CDS to enable dynamic hovering
        row_cds = ColumnDataSource(row)
        # set source to CDS
        # OK to use Gradient for derivative?
        plot.line('date', y_axis, color=color, legend=group, source=row_cds)
    plot.legend.click_policy = "hide"
    plot.legend.location = "top_right"
    hover = plot.select(dict(type=HoverTool))
    # set preffered info for hovertool to show
    hover.tooltips = [
        # need to fix daytime again now....
        ("date", "@date"),
        ("product", "@cm_name"),
        ("Gradient", "@Gradient")
        ]
    
    plots.append(plot)

show(gridplot([plots]))

# Question 2
Do countries in similar regions, also show similar price differences? And if differences occur, can you find a potential explanation?

In [291]:
y_axis = 'Gradient'


plots = []
product_groups = df.groupby(['sub-region', 'cm_name', 'date'])[y_axis].mean().reset_index()
group = product_groups.groupby('sub-region')

for gr1, row1 in group:
    if gr1 == 'Southern Asia':
        plot = figure(x_axis_label="Date", y_axis_label = "Price", x_axis_type="datetime", title="Avg Products in {}".format(gr1), toolbar_location="below", tools='hover,pan,wheel_zoom,box_zoom,reset', plot_width=2800, plot_height=2800)
        product_groups = row1.groupby(['cm_name'])
        for (gr2, row2), color in zip(product_groups, colors):
            row2_cds = ColumnDataSource(row2)
            plot.line('date', y_axis, color=color, legend=gr2, source=row2_cds)
        plot.legend.click_policy = "hide"
        plot.legend.location = "top_right"
        hover = plot.select(dict(type=HoverTool))
        hover.tooltips = [
            ("Product", "@cm_name"),
            ("Value", "@Gradient"),
            ]

        plots.append(plot)
show(gridplot([plots]))

# COVARIANTIE / CORRELATIE 
Olieprijzen en Voedselprijzen

In [345]:
y_axis = 'mp_price'


product_groups = df.groupby(['sub-region', 'cm_name', 'date'])[y_axis].mean().reset_index()
group = product_groups.groupby('sub-region')

for gr1, row1 in group:
    if gr1 == 'Southern Asia':
        print(gr1)
#         date_range = row1[(row1['date'] >= '2016-01-01')]# & (row1['date'] <= '2017-01-01')]
        prod = row1.pivot_table('mp_price', ['sub-region', 'date'], 'cm_name')
        result = prod.corr(method='pearson')
        print(result[['Fuel (diesel)', 'Fuel (petrol-gasoline)']].sort_values('Fuel (diesel)', ascending=False))
#         print(result.groupby(['Fuel (diesel)', 'Fuel (petrol-gasoline)']).mean())



Southern Asia
cm_name                 Fuel (diesel)  Fuel (petrol-gasoline)
cm_name                                                      
Fuel (diesel)                1.000000                0.962579
Fuel (petrol-gasoline)       0.962579                1.000000
Rice                         0.917240                0.795579
Wheat                        0.870644                0.549550
Ghee (artificial)            0.861866                0.809228
Oil (cooking)                0.858851                0.805207
Salt                         0.852428                0.851230
Oil (mustard)                0.852036                0.820398
Oil (vegetable)              0.844108                0.785286
Tea (black)                  0.841504                0.869773
Milk (pasteurized)           0.840138                0.756736
Salt (iodised)               0.827114                0.841970
Onions                       0.807154                0.759228
Oil (sunflower)              0.785020                0.8

# Question 3
Can we see any correlations between other types of data, like: currency fluctuations, weather patterns, and/or refugee movements. Can we perhaps visualize refugee movements from the food price data? Do certain weather conditions influence market prices?

In [3]:
import pandas as pd
import os
import numpy as np

# get datasets
# https://data.world/unhcr/asylum-applications/workspace/file?filename=asylum_seekers_monthly_2016_12_08_152150.csv
asylum_file = 'asylum_seekers_monthly_2016_12_08_152150.csv'
df = pd.read_csv(asylum_file, low_memory=False)

food_prices_file = 'WFPVAM_FoodPrices_version5_Retail.csv'
df_food = pd.read_csv(food_prices_file)


####### pre-processing #######################################################################

df.columns = ['destination', 'origin', 'year', 'month', 'frequency']
# remove abundant rows
df.drop(df.index[:2], inplace=True)

# map month names to corresponding number to allow for convertion to daytime
MONTH_DICT = {'January': 1, 'February': 2, 'March': 3, 'April': 4, 'May': 5, 'June': 6, 
              'July': 7, 'August': 8, 'September': 9, 'October': 10, 'November': 11,
              'December': 12}

df['month'] = df['month'].map(MONTH_DICT)

# month and year column to single daytime column 'date'
df = df.assign(date=pd.to_datetime(df[['year', 'month']].assign(day=1)))
# drop now abundant columns month and year
df = df.drop('month', axis=1)
df = df.drop('year', axis=1)

# drop rows with undefined frequency
df = df.drop(df[df.frequency == '*'].index)
# also drop rows with unknown/various origins
df = df.drop(df[df.origin == 'Various/unknown'].index)

# NB: we could also drop the destination columns, since it only includes countries
# outside of our original dataset. But maybe it will allow for some dynamic hovering later on...
# so we'll keep it for now.

# see if different names are used for identical counrtries in the two databases:
# will end up with a list with countries that don't match; manually see if they should...
countries_food = df_food['Iadm0_name'].unique()
countries_refugee = df['origin'].unique()

possible_mismatches_1 = [country for country in countries_food 
                       if country not in countries_refugee]

possible_mismatches_2 = [country for country in countries_refugee
                        if country not in countries_food]

definite_mismatches = possible_mismatches_1 + possible_mismatches_2

mismatches = definite_mismatches.sort()
# print(mismatches)

# {'country name in refugee file': 'corresponding country name in food dataset'}
MISMATCH_DICT = {'Iran (Islamic Rep. of)': 'Iran  (Islamic Republic of)', 'Dem. Rep. of the Congo': 'Democratic Republic of the Congo',
                 'Central African Rep.': 'Central African Republic'}

# fix mis-matches
for (original, new) in MISMATCH_DICT.items():
    mask = df.origin == original
    df.loc[mask, 'origin'] = new
    


In [2]:
# reset index
df = df.reset_index(drop=True)
df.head()
###################################### pre-processing done ###################################


# set food_prices dataset to daytime to allow merging with refugee dataset
df_food['date'] = pd.to_datetime(df_food.date, format='%Y-%m')
# merge datasets 

new_df = pd.merge(df_food, df,  how='left', left_on=['Iadm0_name','date'], right_on = ['origin','date'])
# drop now abundantan origin row (equals Iadm0_name)
new_df[:50]



,Iadm0_name,mkt_name,cm_name,mp_price,date,um_name,Gradient,destination,origin,frequency
0,Afghanistan,Bamyan,Wheat,4483.786085,2004-04-01,KG,-195.204191,Australia,Afghanistan,3
1,Afghanistan,Bamyan,Wheat,4483.786085,2004-04-01,KG,-195.204191,Austria,Afghanistan,123
2,Afghanistan,Bamyan,Wheat,4483.786085,2004-04-01,KG,-195.204191,Belgium,Afghanistan,25
3,Afghanistan,Bamyan,Wheat,4483.786085,2004-04-01,KG,-195.204191,Bulgaria,Afghanistan,8
4,Afghanistan,Bamyan,Wheat,4483.786085,2004-04-01,KG,-195.204191,Canada,Afghanistan,8
5,Afghanistan,Bamyan,Wheat,4483.786085,2004-04-01,KG,-195.204191,Czech Rep.,Afghanistan,1
6,Afghanistan,Bamyan,Wheat,4483.786085,2004-04-01,KG,-195.204191,Cyprus,Afghanistan,5
7,Afghanistan,Bamyan,Wheat,4483.786085,2004-04-01,KG,-195.204191,Finland,Afghanistan,8
8,Afghanistan,Bamyan,Wheat,4483.786085,2004-04-01,KG,-195.204191,Denmark,Afghanistan,35
9,Afghanistan,Bamyan,Wheat,4483.786085,2004-04-01,KG,-195.204191,France,Afghanistan,13


In [16]:
# also drop rows with unknown/various origins
refugees = refugees.drop(refugees[refugees['Origin'] == 'Various/Unknown'].index)

In [17]:
print(refugees.head(10))

     Year Country / territory of asylum/residence  Origin  \
163  1960                  Dem. Rep. of the Congo  Angola   
179  1961                                 Burundi  Rwanda   
183  1961                  Dem. Rep. of the Congo  Angola   
184  1961                  Dem. Rep. of the Congo  Rwanda   
197  1961                                    Togo   Ghana   
200  1961                 United Rep. of Tanzania  Rwanda   
201  1961                                  Uganda  Rwanda   
204  1962                                 Burundi  Rwanda   
208  1962                  Dem. Rep. of the Congo  Angola   
209  1962                  Dem. Rep. of the Congo  Rwanda   

    Refugees (incl. refugee-like situations) Asylum-seekers (pending cases)  \
163                                   150000                            NaN   
179                                    30000                            NaN   
183                                   150000                            NaN   
184         

# QUESTION 4

In [374]:
y_axis = 'Gradient'


plots = []
group = df.groupby(['sub-region', 'adm0_name', 'cm_name', 'date'])[y_axis].mean().reset_index()
group = group.groupby('sub-region')

for gr1, row1 in group:
    print(gr1)
    if gr1 == 'Sub-Saharan Africa':
        print(gr1)
        product_groups = row1.groupby(['cm_name'])
        for gr3, row3 in product_groups:
            if 'Rice (imported)' in gr3:
                plot = figure(x_axis_label="Date", y_axis_label = "Price", x_axis_type="datetime", title="Avg Products in {}".format(gr3), toolbar_location="below", tools='hover,pan,wheel_zoom,box_zoom,reset')#, plot_width=2800, plot_height=2800)
                country_group = row3.groupby('adm0_name')
                for (gr2, row2), color in zip(country_group, colors):
                    row2_cds = ColumnDataSource(row2)
                    plot.line('date', y_axis, color=color, legend=gr2, source=row2_cds)
                plot.legend.click_policy = "hide"
                plot.legend.location = "top_right"
                hover = plot.select(dict(type=HoverTool))
                hover.tooltips = [
                    ("Product", "@cm_name"),
                    ("Value", "@Gradient"),
                    ]
                show(plot)
#                 plots.append(plot)
                break
        break

# show(gridplot([plots]))

Central Asia
Eastern Europe
Latin America and the Caribbean
Northern Africa
South-eastern Asia
Southern Asia
Sub-Saharan Africa
Sub-Saharan Africa


In [371]:
y_axis = 'mp_price'


product_groups = df.groupby(['sub-region', 'adm0_name', 'cm_name', 'date'])[y_axis].mean().reset_index()
group = product_groups.groupby('adm0_name')

for gr1, row1 in group:
#         date_range = row1[(row1['date'] >= '2016-01-01')]# & (row1['date'] <= '2017-01-01')]
    prod = row1.pivot_table('mp_price', ['adm0_name', 'date'], 'cm_name')
    result = prod.corr(method='pearson')
    
    print(gr1)
    print(result)
#         print(result.groupby(['Fuel (diesel)', 'Fuel (petrol-gasoline)']).mean())



Afghanistan
cm_name                Bread  Fuel (diesel)  Rice (low quality)     Wheat
cm_name                                                                  
Bread               1.000000       0.713070            0.515821  0.860111
Fuel (diesel)       0.713070       1.000000            0.726925  0.762367
Rice (low quality)  0.515821       0.726925            1.000000  0.817250
Wheat               0.860111       0.762367            0.817250  1.000000
Armenia
cm_name                    Apples (red)  Bread (first grade flour)  \
cm_name                                                              
Apples (red)                   1.000000                   0.669704   
Bread (first grade flour)      0.669704                   1.000000   
Bread (high grade flour)       0.659114                   0.993747   
Cabbage                        0.686805                   0.706574   
Carrots                        0.360882                   0.621775   
Cheese (dry)                   0.758137       

cm_name                                         Livestock (goat, medium-sized castrated male)  \
cm_name                                                                                         
Livestock (goat, medium-sized castrated male)                                        1.000000   
Livestock (sheep, medium-sized castrated male)                                       0.538261   
Maize (white)                                                                        0.180431   
Millet                                                                               0.191107   
Rice (imported)                                                                      0.462964   
Rice (local)                                                                         0.139799   
Sorghum (berbere)                                                                    0.271492   
Sorghum (red)                                                                       -0.018831   
Sorghum (white)               

cm_name      Maize flour  Wheat flour
cm_name                              
Maize flour     1.000000     0.626008
Wheat flour     0.626008     1.000000
Guinea
cm_name               Beans (niebe, white)  Bread (Unit)  Cassava meal (gari)  \
cm_name                                                                         
Beans (niebe, white)              1.000000      0.535280             0.793123   
Bread (Unit)                      0.535280      1.000000             0.568914   
Cassava meal (gari)               0.793123      0.568914             1.000000   
Meat (beef)                       0.695211      0.861660             0.842264   
Oil (palm)                        0.583844      0.921003             0.442926   
Rice (imported)                   0.635945      0.760237             0.707772   
Rice (local)                      0.540462      0.758575             0.540837   
Salt                              0.610207      0.920851             0.733901   
Sugar                          

Lao People's Democratic Republic
cm_name                               Eggs  Fish (catfish)  Fish (snake head)  \
cm_name                                                                         
Eggs                              1.000000        0.711358           0.839154   
Fish (catfish)                    0.711358        1.000000           0.932194   
Fish (snake head)                 0.839154        0.932194           1.000000   
Fish (tilapia, farmed)            0.967912        0.687726           0.786515   
Fuel (diesel)                     0.721651        0.669649           0.650651   
Fuel (petrol-gasoline)            0.832002        0.730870           0.749859   
Garlic (small)                    0.934355        0.504785           0.652317   
Meat (beef, first quality)        0.923126        0.785089           0.874515   
Meat (beef, second quality)       0.880303        0.852430           0.910028   
Meat (buffalo, first quality)     0.928165        0.747805           0.87481

Mozambique
cm_name                            Beans (butter)  Beans (catarino)  \
cm_name                                                               
Beans (butter)                           1.000000          0.990470   
Beans (catarino)                         0.990470          1.000000   
Beans (dry)                                   NaN               NaN   
Beans (magnum)                           0.750441          0.730715   
Cassava flour                            0.844476          0.849825   
Cowpeas                                  0.975850          0.976224   
Groundnuts (Mix)                              NaN               NaN   
Groundnuts (large, shelled)              0.740122          0.722979   
Groundnuts (small, shelled)              0.691043          0.654524   
Maize (white)                            0.806572          0.772916   
Maize meal (white, first grade)          0.283585          0.310152   
Maize meal (white, with bran)            0.712857          0.70770

cm_name                          Maize (local)  Meat (chicken)  \
cm_name                                                          
Maize (local)                         1.000000        0.530573   
Meat (chicken)                        0.530573        1.000000   
Oil (vegetable)                       0.691522        0.806457   
Potatoes                              0.103899        0.108427   
Rice (local)                          0.199995        0.738463   
Sugar                                 0.084063        0.414270   
Wheat flour (locally processed)       0.543939        0.860230   

cm_name                          Oil (vegetable)  Potatoes  Rice (local)  \
cm_name                                                                    
Maize (local)                           0.691522  0.103899      0.199995   
Meat (chicken)                          0.806457  0.108427      0.738463   
Oil (vegetable)                         1.000000  0.013406      0.764708   
Potatoes                 

Sudan
cm_name               Millet   Sorghum  Sorghum (food aid)     Wheat
cm_name                                                             
Millet              1.000000  0.977043            0.922790  0.835068
Sorghum             0.977043  1.000000            0.940134  0.756434
Sorghum (food aid)  0.922790  0.940134            1.000000  0.663296
Wheat               0.835068  0.756434            0.663296  1.000000
Tajikistan
cm_name                     Beans (haricot)     Bread   Cabbage   Carrots  \
cm_name                                                                     
Beans (haricot)                    1.000000 -0.549746  0.070981  0.156999   
Bread                             -0.549746  1.000000  0.713632  0.565032   
Cabbage                            0.070981  0.713632  1.000000  0.778412   
Carrots                            0.156999  0.565032  0.778412  1.000000   
Chickpeas                          0.785301 -0.620031  0.345096 -0.132131   
Eggs                          

Yemen
cm_name                 Beans (kidney red)  Beans (white)      Eggs  \
cm_name                                                               
Beans (kidney red)                1.000000       0.952774  0.948882   
Beans (white)                     0.952774       1.000000  0.936582   
Eggs                              0.948882       0.936582  1.000000   
Fuel (diesel)                     0.908134       0.935983  0.943548   
Fuel (gas)                        0.820638       0.828255  0.271239   
Fuel (petrol-gasoline)            0.921478       0.935020  0.968064   
Lentils                           0.958014       0.958151  0.971415   
Oil (vegetable)                   0.864815       0.740578  0.866500   
Onions                            0.912793       0.906037  0.865930   
Peas (yellow, split)              0.958961       0.920980  0.963014   
Potatoes                          0.906159       0.924809  0.945804   
Rice (imported)                   0.954550       0.910007  0.943158   
